In [ ]:
import os.path

import pandas as pd
import os
os.chdir('../')


In [ ]:
from IDV_CS_Model import *
from CS_based_early_stopping import *
import seaborn as sns
import matplotlib.pyplot as plt
from utils import *
import numpy as np
from collections import Counter
import sys
import json
DATA_DIR = '../data/Evaluation_CoTs/Algo_Design_Data/'
# DF_NAME = 'GSM8K'
# DIFFICULTY = 'easy'
# NUM_OF_SAMPLES = 500
# NUM_OF_COT = 40
# MODEL = 'gpt-3.5-turbo-0125'

In [ ]:
file_path = os.path.join(DATA_DIR, 'final_extracted.json')
df_with_features_raw = pd.read_json(file_path, lines=True)


In [ ]:
feature_li =['LEN','QUA_IM','DIF_IV','SIM_INPUT','SIM_COT_BIGRAM','SIM_COT_AGG','SIM_AC_BIGRAM','SIM_AC_PW']
df_with_features_raw = df_with_features_raw[~df_with_features_raw.Model.str.startswith('gpt-4')]

_,df_with_features = train_test_split_stratify(df_with_features_raw,test_size=0.3,random_state =2024)
_, coef = trained_LR_model(df_with_features_raw, feature_li, report_auroc=False,train_mode=True)
print(coef)
coe = coef[1:]
intercept = coef[0]

In [ ]:
df_with_features

In [ ]:
config = {
    'benchmark': ['all','GSM8K_hard','GSM8K_test','MathQA_challenge_test','MathQA_dev','BigBench_hard','BigBench_easy'],
    'llm' : ['all','gpt-3.5','claude-3','llama3'],
    'N': [i for i in range(2,16)],
    'threshold': [0.1*i for i in range(11)],
    'stop_algo':['PositiveN','ConsistencyN'],
    'approximator': [customized_LR_model,trained_LR_model]
}

In [ ]:
def get_data(df,benchmark='all',llm = 'all'):
    if benchmark == 'all':
        df_ = df
    else:
        df_ = df[df.Name == benchmark]
    
    if llm == 'gpt-4':
        df_final = df_[df_.Model.str.startswith('gpt-4')]
    elif llm == 'gpt-3.5':
        df_final = df_[df_.Model.str.startswith('gpt-3')]
    elif llm == 'claude-3':
        df_final = df_[df_.Model.str.startswith('claude-3')]
    elif llm == 'llama3':
        df_final = df_[df_.Model.str.startswith('llama3')]
    else:
        df_final = df_
    return df_final

In [ ]:
for benchmark in config['benchmark']:
    print(len(get_data(df_with_features,benchmark=benchmark)))

# step vs acc main

In [ ]:
result_buffer = {
        'llm': [],
        'benchmark':[],
        'N': [],
        'stop_algo':[],
        'threshold': [],
        'SC_ACC': [],
        'ES_ACC': [],
        'ASC_ACC': [],
        'CASC_ACC': [],
        'SC_Avg_Cost': [],
        'ES_Avg_Cost': [],
        'ASC_Avg_Cost': [],
        'CASC_Avg_Cost': [],
    }
for benchmark in config['benchmark']:
    for llm in config['llm']:
        df_in = get_data(df_with_features,benchmark=benchmark,llm=llm)
        print(llm,benchmark)
        print('df_size is ',len(df_in))
        df_cs = customized_LR_model(df_in,feature_li,coe,intercept)
        N = 3
        threshold = 0.1
        stop_mechanism = 'PositiveN'
        df_results = CS_early_stopping(df=df_cs, threshold=threshold, N=N, stop_mechanism=stop_mechanism)
        df_model_comp_dict = {
                'llm': llm,
                'benchmark':benchmark,
                'N': N,
                'stop_algo':stop_mechanism,
                'threshold': threshold,
                'SC_ACC': round(df_results.SC_correctness.sum() / len(df_results),4),
                'ES_ACC': round(df_results.ES_correctness.sum() / len(df_results),4),
                'ASC_ACC': round(df_results.asc_correctness.sum() / len(df_results),4),
                'CASC_ACC': round(df_results.CS_correctness.sum() / len(df_results),4),
                'SC_Avg_Cost': 40,
                'ES_Avg_Cost': round(df_results.ES_steps.mean(),2),
                'ASC_Avg_Cost': round(df_results.asc_steps.mean(),2),
                'CASC_Avg_Cost': round(df_results.CS_steps.mean(),2),
            }
        for key,val in df_model_comp_dict.items():
            result_buffer[key].append(val)

In [ ]:
df_main_table = pd.DataFrame(result_buffer)
# df_main_table.to_csv('main_table.csv',index=False)
# df_main_table = pd.read_csv('../src/experiment_collection/main_table.csv')
df_main_table

# N study

In [ ]:
result_buffer = {
        'llm': [],
        'benchmark':[],
        'N': [],
        'stop_algo':[],
        'threshold': [],
        'SC_ACC': [],
        'ES_ACC': [],
        'ASC_ACC': [],
        'CASC_ACC': [],
        'SC_Avg_Cost': [],
        'ES_Avg_Cost': [],
        'ASC_Avg_Cost': [],
        'CASC_Avg_Cost': [],
    }
for N in config['N']:
    for benchmark in config['benchmark']:
        for llm in config['llm']:
            df_in = get_data(df_with_features,benchmark=benchmark,llm=llm)
            print(llm,benchmark)
            print('df_size is ',len(df_in))
            df_cs = customized_LR_model(df_in,feature_li,coe,intercept)
            threshold = 0.1
            stop_mechanism = 'PositiveN'
            df_results = CS_early_stopping(df=df_cs, threshold=threshold, N=N, stop_mechanism=stop_mechanism)
            df_model_comp_dict = {
                    'llm': llm,
                    'benchmark':benchmark,
                    'N': N,
                    'stop_algo':stop_mechanism,
                    'threshold': threshold,
                    'SC_ACC': round(df_results.SC_correctness.sum() / len(df_results),4),
                    'ES_ACC': round(df_results.ES_correctness.sum() / len(df_results),4),
                    'ASC_ACC': round(df_results.asc_correctness.sum() / len(df_results),4),
                    'CASC_ACC': round(df_results.CS_correctness.sum() / len(df_results),4),
                    'SC_Avg_Cost': 40,
                    'ES_Avg_Cost': round(df_results.ES_steps.mean(),2),
                    'ASC_Avg_Cost': round(df_results.asc_steps.mean(),2),
                    'CASC_Avg_Cost': round(df_results.CS_steps.mean(),2),
                }
            for key,val in df_model_comp_dict.items():
                result_buffer[key].append(val)
df_N_study = pd.DataFrame(result_buffer)
# df_N_study.to_csv('N_study_table.csv',index=False)

In [ ]:
# df_N_study = pd.read_csv('../src/experiment_collection/N_study_table.csv')
df_N_study

In [ ]:
def replace_zeros_with_previous_nonzero(initialize, arr):
    # Initialize a variable to keep track of the last non-zero value
    last_nonzero = initialize
    
    # Iterate over the array
    for i in range(len(arr)):
        if arr[i] != 0:
            # Update the last non-zero value
            last_nonzero = arr[i]
        else:
            # Replace zero with the last non-zero value
            arr[i] = last_nonzero
            
    return arr
# Create the line plot with solid lines


In [ ]:
llm = 'all'
benchmark = 'all'
print(llm,benchmark)
sc_step_results = []
df_step = get_data(df_with_features,benchmark=benchmark,llm = llm)
for row_idx in range(len(df_step)):
    sample_results = []
    for i in range(40):
        if Counter(df_step['CoT answers'].iloc[row_idx][:i+1]).most_common(1)[0][0] == df_step['correct answer'].iloc[row_idx]:
            sample_results.append(1)
        else:
            sample_results.append(0)
    sc_step_results.append(sample_results)
sc_step_results = np.array(sc_step_results)
sc_step_results = sc_step_results.sum(axis=0)/sc_step_results.shape[0]
df_N_study_gpt4 = df_N_study[(df_N_study.llm == llm)]
df_N_study_gpt4_all = df_N_study_gpt4[df_N_study_gpt4.benchmark == benchmark]
casc_steps = df_N_study_gpt4_all['CASC_Avg_Cost'].to_numpy().astype(np.int32)
casc_acc = df_N_study_gpt4_all['CASC_ACC'].to_numpy()
casc_step_results = np.zeros(40)
for i, idx in enumerate(casc_steps):
    casc_step_results[idx] = casc_acc[i]
casc_step_results = replace_zeros_with_previous_nonzero(sc_step_results[0],casc_step_results)

es_steps = []
es_acc = []
for i in range(16):
    df_es_step = calculate_ES_correctness(df_step,i+1)
    es_acc.append(round(df_es_step.ES_correctness.sum() / len(df_es_step),4))
    es_steps.append(df_es_step.ES_steps.mean().astype(np.int32))
es_step_results = np.zeros(40)
for i, idx in enumerate(es_steps):
    es_step_results[idx] = es_acc[i]
es_step_results = replace_zeros_with_previous_nonzero(sc_step_results[0],es_step_results)

asc_steps = []
asc_acc = []
for i in range(75,101):
    df_asc_step = calculate_ASC_correctness(df_step,i*0.01)
    asc_acc.append(round(df_asc_step.asc_correctness.sum() / len(df_asc_step),4))
    asc_steps.append(df_asc_step.asc_steps.mean().astype(np.int32))
asc_step_results = np.zeros(40)
for i, idx in enumerate(asc_steps):
    asc_step_results[idx-1] = asc_acc[i]
asc_step_results = replace_zeros_with_previous_nonzero(sc_step_results[0],asc_step_results)

step_study_plot_dict = {
    'Steps': np.arange(40)[1:],
    'SC': sc_step_results[1:],
    'ES': es_step_results[1:],
    'ASC': asc_step_results[1:],
    'CASC':casc_step_results[1:]
}



plt.figure()
sns.lineplot(data=step_study_plot_dict, x='Steps', y='SC', label='SC', linestyle='-')
sns.lineplot(data=step_study_plot_dict, x='Steps', y='ES', label='ES', linestyle='-')
sns.lineplot(data=step_study_plot_dict, x='Steps', y='ASC', label='ASC', linestyle='-')
sns.lineplot(data=step_study_plot_dict, x='Steps', y='CASC', label='CASC', linestyle='-')

# Set the x and y axis labels
plt.xlabel('Steps')
plt.ylabel('ACC')

# Find the intersection point
x_intercept = None
for i in range(len(step_study_plot_dict['Steps']) - 1):
    if step_study_plot_dict['CASC'][i] < sc_step_results.max() <= step_study_plot_dict['CASC'][i+1]:
        x_intercept = step_study_plot_dict['Steps'][i]+1
        break

# Add the horizontal line (red line)
plt.hlines(y=0.45, xmin=x_intercept, xmax=40, color='red', linestyle='--', linewidth=1)

# Add the vertical line at the intersection point
if x_intercept is not None:
    plt.vlines(x=x_intercept, ymin=sc_step_results.min(), ymax=0.45, color='red', linestyle='--', linewidth=1)
    plt.annotate(f'({x_intercept}, {0.45})', xy=(x_intercept, 0.45), 
                 xytext=(x_intercept - 5.7, 0.45),
                 fontsize=9, color='black')

# Display the plot
plt.legend()
plt.savefig('../src/experiment_collection/main_step_acc_plot.jpg', dpi=300, bbox_inches='tight')
plt.plot()

In [ ]:
casc_N = df_N_study_gpt4_all['N'].to_numpy()
fig, ax1 = plt.subplots()

# Plot the first line (ACC) with ax1
color = 'tab:blue'
ax1.set_xlabel('N')
ax1.set_ylabel('ACC', color=color)
line1, = ax1.plot(casc_N, casc_acc, color=color, label='ACC')
ax1.tick_params(axis='y', labelcolor=color)
plt.hlines(y=casc_acc[2], xmin=0, xmax=3, color='red', linestyle='--', linewidth=1)
plt.annotate(f'({3}, {casc_acc[2]})', xy=(3, casc_acc[2]), 
                 xytext=(3 - 2, casc_acc[2]+0.001),
                 fontsize=9, color='black')
# Create a second y-axis sharing the same x-axis
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Avg Steps', color=color)
line2, = ax2.plot(casc_N, casc_steps, color=color, label='Avg Steps')
ax2.tick_params(axis='y', labelcolor=color)
plt.vlines(x=3, ymin=0, ymax=22, color='red', linestyle='--', linewidth=1)
plt.hlines(y=casc_steps[2], xmin=3, xmax=14, color='red', linestyle='--', linewidth=1)
plt.annotate(f'({3}, {casc_steps[2]})', xy=(3, casc_steps[2]), 
                 xytext=(3 +0.5, casc_steps[2]-1),
                 fontsize=9, color='black')
# Combine legends
lines = [line1, line2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper left')
fig.savefig('../src/experiment_collection/N_study_plot.jpg', dpi=300, bbox_inches='tight')
# Show the plot
plt.show()

# Threshold Study

In [ ]:
result_buffer = {
        'llm': [],
        'benchmark':[],
        'N': [],
        'stop_algo':[],
        'threshold': [],
        'SC_ACC': [],
        'ES_ACC': [],
        'ASC_ACC': [],
        'CASC_ACC': [],
        'SC_Avg_Cost': [],
        'ES_Avg_Cost': [],
        'ASC_Avg_Cost': [],
        'CASC_Avg_Cost': [],
    }
for threshold in config['threshold']:
    for benchmark in config['benchmark']:
        for llm in config['llm']:
            df_in = get_data(df_with_features,benchmark=benchmark,llm=llm)
            print(llm,benchmark)
            print('df_size is ',len(df_in))
            df_cs = customized_LR_model(df_in,feature_li,coe,intercept)
            N= 3
            stop_mechanism = 'PositiveN'
            df_results = CS_early_stopping(df=df_cs, threshold=threshold, N=N, stop_mechanism=stop_mechanism)
            df_model_comp_dict = {
                    'llm': llm,
                    'benchmark':benchmark,
                    'N': N,
                    'stop_algo':stop_mechanism,
                    'threshold': threshold,
                    'SC_ACC': round(df_results.SC_correctness.sum() / len(df_results),4),
                    'ES_ACC': round(df_results.ES_correctness.sum() / len(df_results),4),
                    'ASC_ACC': round(df_results.asc_correctness.sum() / len(df_results),4),
                    'CASC_ACC': round(df_results.CS_correctness.sum() / len(df_results),4),
                    'SC_Avg_Cost': 40,
                    'ES_Avg_Cost': round(df_results.ES_steps.mean(),2),
                    'ASC_Avg_Cost': round(df_results.asc_steps.mean(),2),
                    'CASC_Avg_Cost': round(df_results.CS_steps.mean(),2),
                }
            for key,val in df_model_comp_dict.items():
                result_buffer[key].append(val)

In [ ]:
df_threshold_study = pd.DataFrame(result_buffer)
# df_threshold_study.to_csv('threshold_study_table.csv',index=False)
# df_threshold_study = pd.read_csv('../src/experiment_collection/threshold_study_table.csv')
# df_threshold_study

In [ ]:
df_threshold_study_gpt4 = df_threshold_study[(df_threshold_study.llm == llm)]
df_threshold_study_gpt4_all = df_threshold_study_gpt4[df_threshold_study_gpt4.benchmark == benchmark]
th_casc_steps = df_threshold_study_gpt4_all['CASC_Avg_Cost'].to_numpy()
th_casc_acc = df_threshold_study_gpt4_all['CASC_ACC'].to_numpy()
th_casc_threshold = df_threshold_study_gpt4_all['threshold'].to_numpy()
# Create a figure and axis
fig, ax1 = plt.subplots()

# Plot the first line (ACC) with ax1
color = 'tab:blue'
ax1.set_xlabel('Threshold')
ax1.set_ylabel('ACC', color=color)
line1, = ax1.plot(th_casc_threshold, th_casc_acc, color=color, label='ACC')
ax1.tick_params(axis='y', labelcolor=color)
plt.hlines(y=th_casc_acc[5], xmin=0, xmax=0.5, color='red', linestyle='--', linewidth=1)
plt.annotate(f'({0.5}, {th_casc_acc[5]})', xy=(0.5, th_casc_acc[5]), 
                 xytext=(0.5 + 0.02, th_casc_acc[5]-0.01),
                 fontsize=9, color='black')
# Create a second y-axis sharing the same x-axis
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Avg Steps', color=color)
line2, = ax2.plot(th_casc_threshold, th_casc_steps, color=color, label='Avg Steps')
ax2.tick_params(axis='y', labelcolor=color)
plt.vlines(x=0.5, ymin=0, ymax=40, color='red', linestyle='--', linewidth=1)
plt.hlines(y=th_casc_steps[1], xmin=0.1, xmax=1, color='red', linestyle='--', linewidth=1)
plt.annotate(f'({0.5}, {6})', xy=(0.5, th_casc_steps[5]), 
                 xytext=(0.5 - 0.1, th_casc_steps[5]+1),
                 fontsize=9, color='black')
# Combine legends
lines = [line1, line2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper left')
plt.savefig("Threshold.pdf", format="pdf", bbox_inches="tight")
# Show the plot
plt.show()

In [ ]:
1

In [ ]:
sns.lineplot( x=th_casc_steps, y=th_casc_acc, label='ACC', linestyle='-')

In [ ]:
th_casc_acc[5]
th_casc_steps[5]

# Stop algo study

In [ ]:
# result_buffer = {
#         'llm': [],
#         'benchmark':[],
#         'N': [],
#         'stop_algo':[],
#         'threshold': [],
#         'SC_ACC': [],
#         'ES_ACC': [],
#         'ASC_ACC': [],
#         'CASC_ACC': [],
#         'SC_Avg_Cost': [],
#         'ES_Avg_Cost': [],
#         'ASC_Avg_Cost': [],
#         'CASC_Avg_Cost': [],
#     }
# for stop_mechanism in config['stop_algo']:
#     for benchmark in config['benchmark']:
#         for llm in config['llm']:
#             df_in = get_data(df_with_features,benchmark=benchmark,llm=llm)
#             print(llm,benchmark)
#             print('df_size is ',len(df_in))
#             df_cs, _ = config['approximator'][0](df=df_in, feature_li=feature_li, coe=coe, intercept=intercept)
#             N= 3
#             threshold = 0.5
#             df_results, _ = CS_early_stopping(df=df_cs, threshold=threshold, N=N, stop_mechanism=stop_mechanism)
#             df_model_comp_dict = {
#                     'llm': llm,
#                     'benchmark':benchmark,
#                     'N': N,
#                     'stop_algo':stop_mechanism,
#                     'threshold': threshold,
#                     'SC_ACC': round(df_results.SC_correctness.sum() / len(df_results),4),
#                     'ES_ACC': round(df_results.ES_correctness.sum() / len(df_results),4),
#                     'ASC_ACC': round(df_results.asc_correctness.sum() / len(df_results),4),
#                     'CASC_ACC': round(df_results.CS_correctness.sum() / len(df_results),4),
#                     'SC_Avg_Cost': 40,
#                     'ES_Avg_Cost': round(df_results.ES_steps.mean(),2),
#                     'ASC_Avg_Cost': round(df_results.asc_steps.mean(),2),
#                     'CASC_Avg_Cost': round(df_results.CS_steps.mean(),2),
#                 }
#             for key,val in df_model_comp_dict.items():
#                 result_buffer[key].append(val)

In [ ]:
# df_stop_algo_study = pd.DataFrame(result_buffer)
# df_stop_algo_study.to_csv('stop_algo_study_table.csv',index=False)
df_N_study_2 = pd.read_csv('../src/experiment_collection/N_study_table_ConsistencyN.csv')
df_N_study = pd.read_csv('../src/experiment_collection/N_study_table.csv')

In [ ]:
llm = 'all'
benchmark = 'all'
print(llm,benchmark)
sc_step_results = []
df_step = get_data(df_with_features,benchmark=benchmark,llm = llm)
for row_idx in range(len(df_step)):
    sample_results = []
    for i in range(40):
        if Counter(df_step['CoT answers'].iloc[row_idx][:i+1]).most_common(1)[0][0] == df_step['correct answer'].iloc[row_idx]:
            sample_results.append(1)
        else:
            sample_results.append(0)
    sc_step_results.append(sample_results)
sc_step_results = np.array(sc_step_results)
sc_step_results = sc_step_results.sum(axis=0)/sc_step_results.shape[0]

df_N_study_gpt4 = df_N_study[(df_N_study.llm == llm)]
df_N_study_gpt4_all = df_N_study_gpt4[df_N_study_gpt4.benchmark == benchmark]
casc_steps = df_N_study_gpt4_all['CASC_Avg_Cost'].to_numpy().astype(np.int32)
casc_acc = df_N_study_gpt4_all['CASC_ACC'].to_numpy()
casc_step_results = np.zeros(40)
for i, idx in enumerate(casc_steps):
    casc_step_results[idx] = casc_acc[i]
casc_step_results = replace_zeros_with_previous_nonzero(sc_step_results[0],casc_step_results)

df_N_study_gpt4_2 = df_N_study_2[(df_N_study_2.llm == llm)]
df_N_study_gpt4_2_all = df_N_study_gpt4_2[df_N_study_gpt4_2.benchmark == benchmark]
casc_steps_2 = df_N_study_gpt4_2_all['CASC_Avg_Cost'].to_numpy().astype(np.int32)
casc_acc_2 = df_N_study_gpt4_2_all['CASC_ACC'].to_numpy()
casc_step_results_2 = np.zeros(40)
for i, idx in enumerate(casc_steps_2):
    casc_step_results_2[idx] = casc_acc_2[i]
casc_step_results_2 = replace_zeros_with_previous_nonzero(sc_step_results[0],casc_step_results_2)

es_steps = []
es_acc = []
for i in range(16):
    df_es_step = calculate_ES_correctness(df_step,i+1)
    es_acc.append(round(df_es_step.ES_correctness.sum() / len(df_es_step),4))
    es_steps.append(df_es_step.ES_steps.mean().astype(np.int32))
es_step_results = np.zeros(40)
for i, idx in enumerate(es_steps):
    es_step_results[idx] = es_acc[i]
es_step_results = replace_zeros_with_previous_nonzero(sc_step_results[0],es_step_results)

asc_steps = []
asc_acc = []
for i in range(75,101):
    df_asc_step = calculate_ASC_correctness(df_step,i*0.01)
    asc_acc.append(round(df_asc_step.asc_correctness.sum() / len(df_asc_step),4))
    asc_steps.append(df_asc_step.asc_steps.mean().astype(np.int32))
asc_step_results = np.zeros(40)
for i, idx in enumerate(asc_steps):
    asc_step_results[idx-1] = asc_acc[i]
asc_step_results = replace_zeros_with_previous_nonzero(sc_step_results[0],asc_step_results)

step_study_plot_dict = {
    'Steps': np.arange(40)[1:],
    'SC': sc_step_results[1:],
    'ES': es_step_results[1:],
    'ASC': asc_step_results[1:],
    'CASC_PositiveN':casc_step_results[1:],
    'CASC_ConsistencyN':casc_step_results_2[1:]
}



plt.figure()
# sns.lineplot(data=step_study_plot_dict, x='Steps', y='SC', label='SC', linestyle='-')
# sns.lineplot(data=step_study_plot_dict, x='Steps', y='ES', label='ES', linestyle='-')
# sns.lineplot(data=step_study_plot_dict, x='Steps', y='ASC', label='ASC', linestyle='-')
sns.lineplot(data=step_study_plot_dict, x='Steps', y='CASC_PositiveN', label='CASC_PositiveN', linestyle='-')
sns.lineplot(data=step_study_plot_dict, x='Steps', y='CASC_ConsistencyN', label='CASC_ConsistencyN', linestyle='-')

# Set the x and y axis labels
plt.xlabel('Steps')
plt.ylabel('ACC')

# Find the intersection point
x_intercept = None
for i in range(len(step_study_plot_dict['Steps']) - 1):
    if step_study_plot_dict['CASC_PositiveN'][i] < sc_step_results.max() <= step_study_plot_dict['CASC_PositiveN'][i+1]:
        x_intercept = step_study_plot_dict['Steps'][i]+1
        break

# Add the horizontal line (red line)
plt.hlines(y=0.545, xmin=x_intercept, xmax=40, color='red', linestyle='--', linewidth=1)

# Add the vertical line at the intersection point
if x_intercept is not None:
    plt.vlines(x=x_intercept, ymin=sc_step_results.min(), ymax=0.545, color='red', linestyle='--', linewidth=1)
    plt.annotate(f'({x_intercept}, {0.545})', xy=(x_intercept, 0.545), 
                 xytext=(x_intercept - 5.7, 0.545),
                 fontsize=9, color='black')

# Display the plot
plt.legend()
plt.savefig('../src/experiment_collection/stop_algo_study_plot.jpg', dpi=300, bbox_inches='tight')
plt.plot()